# Data Analytics SoSe 2021
## Gruppenmitglieder: Anja Stricker, Annika Stadelmann

### Aufgabe 1
#### (Datenvorbereitung)
Lesen Sie den unter der o.g. Adresse verfugbaren Tankstellendatensatz zum Stand 31.05.2021
in einen DataFrame namens df stations ein. Dieser soll in dieser Aufgabe fur die weitere
Verwendung vorbereitet werden.

In [50]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

df_stations = pd.read_csv("2021-05-31-stations.csv")
df_stations.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json
0,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":192,""perio..."
1,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
2,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":31,""period..."
3,e52d755d-4be7-4962-a917-3b0139e6f352,Esso Tankstelle,ESSO,OSNABRUECKER STR. 5,,49163,BOHMTE,52.358250,8.304064,2014-03-18 16:45:31+01,"{""openingTimes"":[{""applicable_days"":31,""period..."
4,e52a5d92-33c6-4c77-b6cd-ea90ca1c7c62,Friedrich Dankelmann,Markant,Strangenhäuschen,10,52070,Aachen,50.801680,6.110960,2014-03-18 16:45:31+01,"{""openingTimes"":[{""applicable_days"":63,""period..."


a) Untersuchen Sie den Datensatz auf fehlende Werte und setzen Sie ggf. eine geeignete Strategie um, um mit diesen zu verfahren. Begrunden Sie Ihr Vorgehen.

In [51]:
df_stations.isnull().sum()

uuid                    0
name                    0
brand                 670
street                  3
house_number         2940
post_code               3
city                    4
latitude                0
longitude               0
first_active            0
openingtimes_json       0
dtype: int64

Im folgenden werden nur noch die Spalten betrachtet die einen NaN Wert über 0 haben.
Alle Spalten mit 0 haben keine NaN Werte und müssen demnach nicht nochmal kontrolliert werden.

NaN Werte werden durch passende andere Werte ersetzt.

In [52]:
df_stations["brand"] = df_stations["brand"].fillna("other")
df_stations["openingtimes_json"] = df_stations["openingtimes_json"].replace("{}", "empty")

In [53]:
df_stations.isnull().sum()

uuid                    0
name                    0
brand                   0
street                  3
house_number         2940
post_code               3
city                    4
latitude                0
longitude               0
first_active            0
openingtimes_json       0
dtype: int64

Jetzt werden fehlende Einträge von street, house_number, city und post_code mit geopy ermittelt und ersetzt.

In [5]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="DA_stud")
# df = pd.DataFrame()

# for i in df_stations.loc[df_stations["street"].isnull() | df_stations["house_number"].isnull() | df_stations["city"].isnull() | df_stations["post_code"].isnull()].index:
#     data = df_stations.iloc[i]
#     if ((data["latitude"] <= 90.0) & (data["latitude"] >= -90.0) & (data["longitude"] <= 180.0) & (data["longitude"] >= -180.0) 
#     & (data["latitude"] != 0.0) & (data["longitude"] != 0.0)):
#         location = geolocator.reverse(str(data["latitude"]) + ", " + str(data["longitude"]))
#         adressData = location.raw["address"]
#         if "road" in adressData:
#             df_stations.at[i, "street"] = adressData["road"]
#         if "house_number" in adressData:
#             df_stations.at[i, "house_number"] = adressData["house_number"]
#         if "town" in adressData:
#             df_stations.at[i, "city"] = adressData["town"]
#         if "postcode" in adressData:
#             df_stations.at[i, "post_code"] = adressData["postcode"]

# df_stations[df_stations["city"].isnull()]

In [54]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="DA_stud")
df = pd.DataFrame()

for i in tqdm(df_stations.loc[df_stations["street"].isnull() | df_stations["city"].isnull() | df_stations["post_code"].isnull()].index):
    data = df_stations.iloc[i]
    if ((data["latitude"] <= 90.0) & (data["latitude"] >= -90.0) & (data["longitude"] <= 180.0) & (data["longitude"] >= -180.0) 
    & (data["latitude"] != 0.0) & (data["longitude"] != 0.0)):
        location = geolocator.reverse(str(data["latitude"]) + ", " + str(data["longitude"]))
        adressData = location.raw["address"]
        if "road" in adressData:
            df_stations.at[i, "street"] = adressData["road"]
        if "town" in adressData:
            df_stations.at[i, "city"] = adressData["town"]
        if "postcode" in adressData:
            df_stations.at[i, "post_code"] = adressData["postcode"]

100%|██████████| 6/6 [00:02<00:00,  2.65it/s]


In [55]:
for i in tqdm(df_stations.loc[df_stations["house_number"].isnull()].index):
    data = df_stations.iloc[i]
    if ((data["latitude"] <= 90.0) & (data["latitude"] >= -90.0) & (data["longitude"] <= 180.0) & (data["longitude"] >= -180.0) 
    & (data["latitude"] != 0.0) & (data["longitude"] != 0.0)):
        location = geolocator.reverse(str(data["latitude"]) + ", " + str(data["longitude"]))
        adressData = location.raw["address"]
        if "house_number" in adressData:
            df_stations.at[i, "house_number"] = adressData["house_number"]

100%|██████████| 2940/2940 [24:29<00:00,  2.00it/s]


In [1]:
df_stations["street"] = df_stations["street"].fillna("")
df_stations["post_code"] = df_stations["post_code"].fillna("")
df_stations["city"] = df_stations["city"].fillna("")

SyntaxError: invalid non-printable character U+00A0 (<ipython-input-1-5ec2e95185d4>, line 1)

In [56]:
for i in tqdm(df_stations.loc[(df_stations["latitude"] == 0.0) & (df_stations["longitude"] == 0.0)].index):
    data = df_stations.iloc[i]

    if ((data["street"] == "") & (data["post_code"] == "") & (data["city"] == "")):

        #address we need to geocode
        loc = data["street"] + data["post_code"] + data["city"]
        
        #applying geocode method to get the location
        location = geolocator.geocode(loc)
        
        #printing address and coordinates
        print(location.address)
        print((location.latitude, location.longitude))

        if "house_number" in adressData:
            df_stations.at[i, "latitude"] = location.latitude
            df_stations.at[i, "longitude"] = location.longitude

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'notnull'

In [ ]:
df_stations.head()

In [ ]:
df_stations.drop(df_stations.loc[(df_stations["latitude"] == 0.0) & (df_stations["longitude"] == 0.0)], axis=1)

df

Prüfen ob Cities NaN sind

Einträge die City, Post_Code und Coordinaten NaN haben werden gelöscht.

Citys die einen NaN Wert haben, aber einen Postcode, werden mit den anderen Postcode einträgen verglichen und erhalten den Namen der City.

In [43]:
def findCityWithPostcode(row):
    test = df_stations["city"][(df_stations["post_code"] == row["post_code"]) & (df_stations["city"].notnull())].item()
    if (len(test) > 0):
        return test
    else
        return ""
df_stations["city"] = df_stations[["city", "post_code"]].apply(lambda x: findCityWithPostcode(x) if x["city"] is np.nan else x["city"], axis=1)

SyntaxError: invalid syntax (<ipython-input-43-306eed9bfc74>, line 3)

In [ ]:
df_stations["house_number"] = df_stations["house_number"].fillna(0)
df_stations = df_stations.dropna(subset=["city"])
df_stations.isnull().sum()

In [46]:
df_stations.to_csv("df_stations.csv", index=True)

b) Transformieren Sie die Spalten brand, street und city, sodass die Werte jeweils nur
in Großbuchstaben ausgeben werden.

In [ ]:
df_stations["brand"] = df_stations["brand"].str.upper()
df_stations["street"] = df_stations["street"].str.upper()
df_stations["city"] = df_stations["city"].str.upper()

c) Uberprüfen Sie, ob der Datensatz Duplikate enth ¨ alt und entfernen Sie diese ggf. Ein Dupli- ¨
kat liege dann vor, wenn zwei Tankstellen in den Spalten name, street, house number
und post code ubereinstimmen. Geben Sie aus, wie viele Duplikate ermittelt und entfernt ¨
wurden.

In [ ]:
df_stations[["name", "street", "house_number", "post_code"]].duplicated().sum()

In [ ]:
#df_stations.duplicated(subset = {"name", "street", "house_number", "post_code"}, keep = "first") 

df_stations.drop_duplicates(subset = {"name", "street", "house_number", "post_code"}, keep =  "first", inplace = True)
df_stations[["name", "street", "house_number", "post_code"]].duplicated().sum()

Es wurden 52 Duplikate ermittelt

In [82]:
# df_stations[["name", "street", "house_number", "post_code"]] = df_stations[["name", "street", "house_number", "post_code"]].drop_duplicates()

d) Untersuchen Sie die Spalten longitude und latitude auf unphysikalische Werte, indem Sie zunachst deren Verteilung in einem Histogramm visualisieren. Entfernen oder kor- ¨
rigieren Sie anschließend die Eintrage zu denjenigen Tankstellen, bei denen Sie unphysi- ¨
kalische Werte festgestellt haben. Zeichnen Sie anschließend erneut Histogramme fur die ¨
beiden Spalten longitude und latitude.

Zuerst analysieren wir die Daten.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_stations["latitude"], 50, label='latitude')
plt.hist(df_stations["longitude"], 50, label='longitude')
plt.legend(loc='upper right')
plt.title("Geo Data")
plt.xlabel('coordinates') 
plt.ylabel('count')
plt.show()

Bei Longitude ist ein Ausreißer beim Wert 0 zu erkennen. Diese werden im Folgenden gefiltert.
Ebenso sind unphysikalische Werte zu erkennen, welche ebenfalls korrigiert werden.

In [ ]:
df_stations = df_stations.loc[(df_stations["longitude"] != 0.0) & (df_stations["latitude"] != 0.0)]
df_stations = df_stations.loc[(df_stations["latitude"] <= 90.0) & (df_stations["latitude"] >= -90.0) & (df_stations["longitude"] <= 180.0) & (df_stations["longitude"] >= -180.0)]

In [ ]:
plt.hist(df_stations["latitude"], 10, label='latitude')
plt.hist(df_stations["longitude"], 50, label='longitude')
plt.legend(loc='upper right')
plt.title("Geo Data")
plt.xlabel('coordinates') 
plt.ylabel('count')
plt.show()

Jetzt sehen wir die korrekte Koordinaten Daten

e) Reichern Sie den Tankstellendatensatz um die Spalte state an, die das Bundesland enthalten soll, in dem sich die Tankstelle befindet. Erzeugen Sie durch Filterung einen DataFrame
namens df_stations_BY, der die Informationen zu allen Tankstellen in Bayern enthalt.

In [ ]:
df_stations = df_stations.reset_index(drop=True)
df_stations.shape[0]

In [53]:
df_stations[-10:].index

RangeIndex(start=16241, stop=16251, step=1)

In [74]:
# for i in tqdm(df_stations.index):
#     data = df_stations.iloc[i]
#     location = geolocator.reverse(str(data["latitude"]) + ", " + str(data["longitude"]))
#     adressData = location.raw["address"]
#     if "state" in adressData:
#         df_stations.at[i, "state"] = adressData["state"]

# df_stations["state"]

100%|██████████| 10/10 [00:05<00:00,  1.67it/s]


0                     Bayern
1                     Hessen
2        Nordrhein-Westfalen
3              Niedersachsen
4        Nordrhein-Westfalen
                ...         
16286     Schleswig-Holstein
16287          Niedersachsen
16288          Niedersachsen
16289          Niedersachsen
16290    Nordrhein-Westfalen
Name: state, Length: 16291, dtype: object

In [ ]:
df_bundesland_plz = pd.read_csv("zuordnung_plz_ort.csv")
df_bundesland_plz.head()

In [ ]:
plz_bundesland_dict = {}

for i, j in zip(df_bundesland_plz.plz, df_bundesland_plz.bundesland):
    plz_bundesland_dict[str(i)] = j
    

In [ ]:
df_stations["state"] = df_stations["post_code"].map(plz_bundesland_dict)
df_stations.isnull().sum()

In [ ]:
for i in tqdm(df_stations.loc[df_stations["state"].isnull()].index):
    data = df_stations.iloc[i]
    location = geolocator.reverse(str(data["latitude"]) + ", " + str(data["longitude"]))
    adressData = location.raw["address"]
    if "state" in adressData:
        df_stations.at[i, "state"] = adressData["state"]

df_stations["state"].isnull().sum()

In [ ]:
df_stations[df_stations["state"].isnull()]

In [ ]:
df_stations.to_csv("State_checkpoint.csv", index=False)

In [ ]:
df_stations = pd.read_csv("State_checkpoint.csv")

In [ ]:
df_stations_BY = df_stations.loc[df_stations["state"] == "Bayern"]

Lesen Sie nun den Preise-Datensatz aus dem GitLab-Repository in einen DataFrame namens
df_prices_BY ein. Dieser wurde durch Filterung der auf der Tankerkonig-Seite verf ¨ ugbaren ¨
Preise-Datensatze erzeugt und enth ¨ alt die Angaben zur historischen Preisen und Preis ¨ anderungen ¨
an den Tankstellen in Bayern im Betrachtungszeitraum zwischen Januar 2018 und Mai 2021.

In [ ]:
df_prices_BY = pd.read_csv("prices_BY.csv")

f) Untersuchen Sie die Kraftstoffpreise auf Ausreißer und entfernen Sie diese ggf.

In [ ]:
df_prices_BY.head()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_prices_BY["diesel"], 50, label='diesel')
plt.hist(df_prices_BY["e5"], 50, label='e5')
plt.hist(df_prices_BY["e10"], 50, label='e10')
plt.legend(loc='upper right')
plt.title("Geo Data")
plt.xlabel('coordinates') 
plt.ylabel('count')
plt.show()

In [ ]:
df_prices_BY["e10"] = df_prices_BY["e10"].replace(0.000, np.nan)
df_prices_BY.loc[df_prices_BY["e10"] < 0.5] = np.nan
df_prices_BY["e5"] = df_prices_BY["e5"].replace(0.000, np.nan)
df_prices_BY["diesel"] = df_prices_BY["diesel"].replace(0.000, np.nan)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_prices_BY["diesel"], 30, label='diesel')
plt.hist(df_prices_BY["e5"], 30, label='e5')
plt.hist(df_prices_BY["e10"], 30, label='e10')
plt.legend(loc='upper right')
plt.title("Geo Data")
plt.xlabel('coordinates') 
plt.ylabel('count')
plt.show()

g) Uberf ¨ uhren Sie die Spalte ¨ date in ein DateTime-Format.

In [ ]:
df_prices_BY["date"] = pd.to_datetime(df_prices_BY["date"], infer_datetime_format=True, utc=True)
df_prices_BY["date"].head()

### Aufgabe 2 
#### (Explorative Datenanalyse)
In dieser Aufgabe sind erste einfache Analysen durchzufuhren und Visualisierungen zu erstellen

a) Wie viele Tankstellen gibt es insgesamt in Deutschland? Wie viele pro Bundesland?

In [ ]:
# tankstellenZahl = len(df_stations)
# tankstellenZahl

# need state df

haeufigkeitTankstellen = {"Deutschland": 0, "Baden-Wuerttemberg": 0, "Bayern": 0, "Berlin": 0, "Brandenburg": 0, "Bremen": 0, "Hamburg": 0, "Hessen": 0, "Mecklenburg-Vorpommern": 0, "Niedersachsen": 0, "Nordrhein-Westfalen": 0, "Rheinland-Pfalz": 0, "Saarland": 0, "Sachsen": 0, "Sachsen-Anhalt": 0, "Schleswig-Holstein": 0, "Thueringen": 0, "other": 0}

for x in df_stations["state"]:
    if x == "Baden-Württemberg":
        haeufigkeitTankstellen["Baden-Wuerttemberg"] += 1
    if x == "Bayern":
        haeufigkeitTankstellen["Bayern"] += 1
    if x == "Berlin":
        haeufigkeitTankstellen["Berlin"] += 1
    if x == "Brandenburg":
        haeufigkeitTankstellen["Brandenburg"] += 1
    if x == "Bremen":
        haeufigkeitTankstellen["Bremen"] += 1
    if x == "Hamburg":
        haeufigkeitTankstellen["Hamburg"] += 1
    if x == "Hessen":
        haeufigkeitTankstellen["Hessen"] += 1
    if x == "Mecklenburg-Vorpommern":
        haeufigkeitTankstellen["Mecklenburg-Vorpommern"] += 1
    if x == "Niedersachsen":
        haeufigkeitTankstellen["Niedersachsen"] += 1
    if x == "Nordrhein-Westfalen":
        haeufigkeitTankstellen["Nordrhein-Westfalen"] += 1
    if x == "Rheinland-Pfalz":
        haeufigkeitTankstellen["Rheinland-Pfalz"] += 1
    if x == "Saarland":
        haeufigkeitTankstellen["Saarland"] += 1
    if x == "Sachsen":
        haeufigkeitTankstellen["Sachsen"] += 1
    if x == "Sachsen-Anhalt":
        haeufigkeitTankstellen["Sachsen-Anhalt"] += 1
    if x == "Schleswig-Holstein":
        haeufigkeitTankstellen["Schleswig-Holstein"] += 1
    if x == "Thüringen":
        haeufigkeitTankstellen["Thueringen"] += 1
    if x == "other":
        haeufigkeitTankstellen["other"] += 1
    haeufigkeitTankstellen["Deutschland"] += 1

haeufigkeitTankstellen
# df_stations.isnull().sum()

b) Welches sind die zehn Tankstellenmarken mit den meisten Niederlassungen in Deutschland?
Wie viele Niederlassungen sind es jeweils?

In [ ]:
meisteMarken = list(df_stations['brand'].value_counts()[0:10].index)

def Convert(lst):
    res_dct = {lst[i]: 0 for i in range(0, len(lst), 1)}
    return res_dct

lst = Convert(meisteMarken)

for x in df_stations["brand"]:
    if x in lst:
        lst[x] += 1

lst

c) Welcher Anteil der Tankstellen hat sonntags geoffnet?

In [ ]:
df_stations["openingtimes_json"]

openingTimes = list(df_stations['openingtimes_json'])
openOnSunday = {"Sonntags_geöffnet": 0, "Sonntags_geschlossen": 0}

def Convert(lst):
    res_dct = {lst[i]: 0 for i in range(0, len(lst), 1)}
    return res_dct

lst = Convert(openingTimes)

for x in df_stations["openingtimes_json"]:
    if x["applicable_days"] << 6:
        openOnSunday["Sonntags_geöffnet"] += 1
    else:
        openOnSunday["Sonntags_geschlossen"] += 1

openOnSunday

d) Visualisieren Sie in einem Kreisdiagramm die Zusammensetzung des Marktes in freie Tankstellen und Markentankstellen. Nehmen Sie vereinfachend an, dass eine Tankstelle dann
eine freie Tankstelle ist, wenn sie gemaß der Spalte ¨ brand einer Marke angehort, die bun- ¨
desweit hochstens 20 Niederlassungen betreibt.

In [ ]:
brand = list(df_stations['brand'].value_counts().index)

def Convert(lst):
    res_dct = {lst[i]: 0 for i in range(0, len(lst), 1)}
    return res_dct

lst = Convert(brand)

for x in df_stations["brand"]:
    if x in lst:
        lst[x] += 1

tankstellenListe = {"Markentankstellen": 0, "Freie Tankstellen": 0}

for x in lst.values():
    if x > 20:
        tankstellenListe["Markentankstellen"] += x
    else:
        tankstellenListe["Freie Tankstellen"] += x
tankstellenListe

In [ ]:
def pie_label_formatter(x):
    return '{:.2f}%'.format(x)

plt.pie(tankstellenListe.values(), labels = tankstellenListe.keys(), autopct=pie_label_formatter)
plt.title("Anzahl der freien und Markentankstellen")
plt.show()

e) Wie haufig wird an den bayerischen Tankstellen durchschnittlich pro Tag im Betrachtungs- ¨
zeitraum ein Datensatz an die Markttransparenzstelle fur Kraftstoffe ¨ ubermittelt? 

In [ ]:
test = df_prices_BY["date"].groupby(df_prices_BY["date"].dt.date).count()
test

In [ ]:
test.mean()

f) In welchen zehn bayerischen Stadten war der durchschnittliche Diesel-Preis im Betrach- ¨
tungszeitraum 2018-2021 am hochsten und wie hoch war er jeweils? In welchen zehn ¨
Stadten war er am geringsten und wie hoch war er jeweils? 

In [37]:
# TODO

g) Erstellen Sie mit folium eine interaktive Karte, auf der die einzelnen Tankstellen in Bayern
als Marker (in einem Marker-Cluster) eingezeichnet sind. Beim Klick auf einen Marker soll
der Name der Tankstelle und deren Adresse angezeigt werden.

In [38]:
#TODO

### Aufgabe 3
#### (Analyse der historischen Entwicklung der Kraftstoffpreise)

a) In welchem Monat im Betrachtungszeitraum waren die Kraftstoffpreise fur Diesel in Bay- ¨
ern durchschnittlich am hochsten? In welchem am geringsten? Wie hoch waren die durch- ¨
schnittlichen Preise jeweils?

In [39]:
#TODO

b) Ermitteln Sie die Jahresdurchschnitte fur die einzelnen Kraftstoffsorten in den Jahren 2018- ¨
2021 fur die bayerischen Tankstellen. Visualisieren Sie diese in einem geeigneten Dia- ¨
gramm. Lassen sich Trends erkennen?

In [40]:
#TODO

c) Visualisieren Sie auf geeignete Weise den zeitlichen Verlauf der Preise fur die Kraftstoffe ¨
Diesel und Super E5 und Super E10 an den bayerischen Tankstellen zwischen 2018 und
2021. Beschreiben und interpretieren Sie Ihre Beobachtungen.

In [41]:
#TODO

### Aufgabe 4
#### (Abhangigkeit zwischen dem Roh ¨ olpreis und den Kraftstoffpreisen)

a) Untersuchen Sie jeweils den Zusammenhang zwischen den Kraftstoffpreisen fur Diesel bzw. ¨
Super E5 und dem Roholpreis. Berechnen Sie dazu die Tagesmittelwerte f ¨ ur die Kraftstoff- ¨
preise fur alle Tage im Betrachtungszeitraum 2018-2021 (f ¨ ur die bayerischen Tankstellen) ¨
und stellen Sie diese jeweils den Tagesschlusskursen fur die Roh ¨ olsorte Brent Crude Oil ¨
(offentlich verf ¨ ugbar) in zwei Streudiagrammen (eines f ¨ ur Diesel und eines f ¨ ur E5) ge- ¨
genuber. Laden Sie die verwendeten ¨ Olpreis-Daten in Form einer CSV-Datei mit Ihrer Ab- ¨
gabe auf Moodle hoch.

In [42]:
#TODO

b) Berechnen Sie jeweils fur Diesel und E5 den empirischen Korrelationskoeffizienten zwi- ¨
schen den Roholpreisen und den Tagesmittelwerten der Kraftstoffpreise.

In [43]:
#TODO

c) Diskutieren Sie, ob lineare Modelle sinnvoll waren, um die Zusammenh ¨ ange zwischen dem ¨
Roholpreis und den Preisen f ¨ ur Diesel bzw. E5 darzustellen.

In [44]:
#TODO

### Aufgabe 5
#### (Einfluss des Wettbewerbs auf die Kraftstoffpreise in Bayern)
Untersuchen Sie, welchen Einfluss der Wettbewerb zwischen mehreren Tankstellen auf die Preisgestaltung hat. Ermitteln Sie dazu zunachst f ¨ ur jede Tankstelle in Bayern, wie viele Tankstellen ¨
sich im Umkreis von 5km Umgebung zu ihr befindet. Speichern Sie diese Angabe in einer Spalte namens neighbors. Speichern Sie den resultierenden DataFrame df stations BY als
CSV-Datei namens stations BY.csv und laden Sie diese mit Ihrer Abgabe auf Moodle hoch.
Stellen Sie zunachst m ¨ ogliche Hypothesen auf und untersuchen Sie diese anschließend, indem Sie ¨
geeignete Auswertungen und Visualisierungen erstellen.

In [45]:
#TODO

### Aufgabe 6
#### (Tanken in Amberg, Interaktives Diagramm)
Herr Sparfuchs, der als Rentner zeitlich flexibel ist, mochte wissen, wann und wo er in Am- ¨
berg derzeit am gunstigsten tanken kann. Erstellen Sie dazu ein geeignetes interaktives Dia- ¨
gramm mit Plotly, um die Kraftstoffpreise an den Tankstellen in Amberg zu analysieren, und
leiten Sie Empfehlungen fur Herrn Sparfuchs ab. Erstellen Sie f ¨ ur die Analyse zwei DataFrames ¨
namens df stations amberg und df prices amberg, die aus Filterung der Tankstellenund Preisdatensatze hervorgehen, und verwenden Sie ausschließlich diese f ¨ ur die Erzeugung des ¨
Plotly-Diagramms. Dokumentieren Sie Ihre Analyseergebnisse.

In [46]:
#TODO